# UK House Prices

-----

### Contents

* <a href="#load"> Load and Clean Data</a>
* <a href="#scen1"> Scenario -  Buy in Jan 2005, sell in Dec 2015</a>
    * Plot 1 - <a href="#plot1"> Regional Scatter Plot of return vs volatility </a>
    * Plot 2 - <a href="#plot2"> Top 10 Performers over time </a>
* <a href="#scen2"> Regression - Average monthly returns over period Jan 05 - Jan 2015</a>
    * Plot 1 - <a href="#scen2plot1"> Regional Scatter Plot of return vs volatility </a>
    * Plot 2 - <a href="#scen2plot2"> Top 10 Performers over time </a>


#### Load Necessary Libraries

In [1]:
import pandas as pd
import numpy as np 
from scipy import stats
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
import bokeh.charts
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

<a id='load'></a>
## Load in HPI data and clean

In [13]:
df = pd.read_csv("http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/UK-HPI-full-file-2016-05.csv", 
                 parse_dates=True)
df.head()

,Date,RegionName,AreaCode,AveragePrice,Index,IndexSA,1m%Change,12m%Change,AveragePriceSA,SalesVolume,...,FOO1m%Change,FOO12m%Change,NewPrice,NewIndex,New1m%Change,New12m%Change,OldPrice,OldIndex,Old1m%Change,Old12m%Change
0,01/01/2004,Aberdeenshire,S12000034,81693.66964,40.864214,NaN,NaN,NaN,NaN,388.0,...,NaN,NaN,88436.13953,40.267250,NaN,NaN,81043.95084,40.883367,NaN,NaN
1,01/02/2004,Aberdeenshire,S12000034,81678.76231,40.856757,NaN,-0.018248,NaN,NaN,326.0,...,NaN,NaN,88606.44649,40.344795,0.192576,NaN,80965.29542,40.843688,-0.097053,NaN
2,01/03/2004,Aberdeenshire,S12000034,83525.09702,41.780317,NaN,2.260483,NaN,NaN,453.0,...,NaN,NaN,90296.91375,41.114508,1.907838,NaN,82903.23948,41.821302,2.393549,NaN
3,01/04/2004,Aberdeenshire,S12000034,84333.67900,42.184780,NaN,0.968071,NaN,NaN,571.0,...,NaN,NaN,90319.87844,41.124964,0.025432,NaN,84003.99161,42.376586,1.327755,NaN
4,01/05/2004,Aberdeenshire,S12000034,86379.95396,43.208353,NaN,2.426403,NaN,NaN,502.0,...,NaN,NaN,91989.17763,41.885039,1.848208,NaN,86222.73484,43.495852,2.641235,NaN


#### Make column names lower case for ease of referencing them

In [15]:
df.columns = df.columns.str.lower()
df.dtypes

date                       object
regionname                 object
areacode                   object
averageprice              float64
index                     float64
indexsa                   float64
1m%change                 float64
12m%change                float64
averagepricesa            float64
salesvolume               float64
detachedprice             float64
detachedindex             float64
detached1m%change         float64
detached12m%change        float64
semidetachedprice         float64
semidetachedindex         float64
semidetached1m%change     float64
semidetached12m%change    float64
terracedprice             float64
terracedindex             float64
terraced1m%change         float64
terraced12m%change        float64
flatprice                 float64
flatindex                 float64
flat1m%change             float64
flat12m%change            float64
cashprice                 float64
cashindex                 float64
cash1m%change             float64
cash12m%change

#### Check Data Types

In [16]:
df.dtypes

date                       object
regionname                 object
areacode                   object
averageprice              float64
index                     float64
indexsa                   float64
1m%change                 float64
12m%change                float64
averagepricesa            float64
salesvolume               float64
detachedprice             float64
detachedindex             float64
detached1m%change         float64
detached12m%change        float64
semidetachedprice         float64
semidetachedindex         float64
semidetached1m%change     float64
semidetached12m%change    float64
terracedprice             float64
terracedindex             float64
terraced1m%change         float64
terraced12m%change        float64
flatprice                 float64
flatindex                 float64
flat1m%change             float64
flat12m%change            float64
cashprice                 float64
cashindex                 float64
cash1m%change             float64
cash12m%change

Date should be a datetime type but is interpreted as a string
#### Set Date column type to datetime

In [17]:
df.date = pd.to_datetime(df.date)
df.dtypes

date                      datetime64[ns]
regionname                        object
areacode                          object
averageprice                     float64
index                            float64
indexsa                          float64
1m%change                        float64
12m%change                       float64
averagepricesa                   float64
salesvolume                      float64
detachedprice                    float64
detachedindex                    float64
detached1m%change                float64
detached12m%change               float64
semidetachedprice                float64
semidetachedindex                float64
semidetached1m%change            float64
semidetached12m%change           float64
terracedprice                    float64
terracedindex                    float64
terraced1m%change                float64
terraced12m%change               float64
flatprice                        float64
flatindex                        float64
flat1m%change   

----
<a id ="scen1"></a>
## Scenario 1 - Buying in Jan 2005 and selling Dec 2015

#### Get the value of the index at the start date by region

In [18]:
start_index = df[df["date"] == "01/01/2005"][["regionname","index"]]
start_index.name = "start_index"
start_index.head()

,regionname,index
12,Aberdeenshire,50.931127
269,Adur,73.293675
526,Allerdale,84.304274
783,Amber Valley,87.870328
932,Angus,57.746757


#### Get the value of the index at the end date by region

In [19]:
end_index = df[df["date"] == "01/12/2015"][["regionname","index"]]
end_index.name = "end_index"
end_index.head()

,regionname,index
143,Aberdeenshire,99.226738
400,Adur,110.060075
657,Allerdale,101.360195
914,Amber Valley,105.591544
1063,Angus,101.159727


#### Merge the two dataframes

In [20]:
combined = start_index.merge(end_index, on="regionname", suffixes=('_start','_end'))

#### Calculate the percentage increase between the two

In [29]:
combined["return"] = combined["index_end"].div(combined["index_start"])

#### Sort return by highest % increase first

In [23]:
combined.sort_values("return", ascending=False).head(10)

,regionname,index_start,index_end,return
195,Kensington And Chelsea,36.392068,96.311210,2.646489
85,City of Westminster,40.331205,103.764747,2.572815
53,Camden,44.838339,103.513355,2.308590
163,Hackney,49.117001,112.669236,2.293895
166,Hammersmith and Fulham,45.990206,105.406709,2.291938
347,Southwark,48.977835,109.268629,2.230981
189,Inner London,49.462572,108.052308,2.184527
194,Islington,47.663741,103.589510,2.173340
399,Wandsworth,49.564422,107.259016,2.164032
81,City of London,42.156075,91.014872,2.158998


#### Calculate standard deviation as a measure of volatility

In [36]:
volatility = df.groupby("regionname").std()
volatility = volatility.reset_index().loc[:,["regionname", "index"]]
volatility = volatility.rename(columns={"index":"stdev"})
volatility.sort_values("stdev", ascending=True).head(10)

,regionname,stdev
391,United Kingdom,7.236650
157,Great Britain,8.275733
125,East Renfrewshire,8.400661
356,Stirling,8.954317
79,City of Glasgow,9.025435
118,East Dunbartonshire,9.162110
78,City of Edinburgh,9.519585
325,South Ayrshire,9.604926
146,Fife,10.618393
112,Dumfries and Galloway,10.622473


#### Merge risk and return together

In [37]:
return_risk = volatility.merge(combined, left_on="regionname", right_on="regionname",
                                      suffixes=("_std","_return"))

return_risk = return_risk[["regionname", "stdev", "return"]]
return_risk = return_risk.rename(columns={"stdev":"risk"})
return_risk.head()

,regionname,risk,return
0,Aberdeenshire,16.742720,1.948253
1,Adur,26.851065,1.501631
2,Allerdale,30.943465,1.202314
3,Amber Valley,28.439592,1.201675
4,Angus,14.303827,1.751782


<a id="plot1"></a>
### Plot 1 - Risk versus Return by Region

In [39]:
hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("desc", "@desc"),
        ])
source = ColumnDataSource(
        data=dict(
            x=return_risk["risk"],
            y=return_risk["return"],
            desc=return_risk["regionname"]
        )
    )

p = figure(plot_width=800, plot_height=600, tools=[hover])

p.xaxis.axis_label = "Risk (std dev)"
p.yaxis.axis_label = "Return (%increase since 2005)"

p.circle('x', 'y', source=source)

show(p)

In [51]:
top_regions = return_risk.sort_values("return", ascending=False).iloc[:10,0]
top_regions_data = df[df["regionname"].isin(top_regions)]
top_regions_index = top_regions_data.pivot(index="date",columns="regionname", values="index")
top_regions_index.head()

regionname,Camden,City of London,City of Westminster,Hackney,Hammersmith and Fulham,Inner London,Islington,Kensington And Chelsea,Southwark,Wandsworth
date,,,,,,,,,,
1995-01-01,15.711197,11.669374,13.620584,13.058748,17.054761,15.266276,15.011064,13.910526,15.180262,15.989986
1995-01-02,15.526192,10.489509,13.461165,13.461517,16.670370,14.804637,15.307320,13.883908,14.490150,16.004787
1995-01-03,15.626663,10.096220,13.542262,13.548058,16.472139,14.942351,15.165103,13.924535,14.705769,15.731025
1995-01-04,15.602718,9.838522,13.655902,13.877495,16.579828,14.993069,15.145411,14.023373,14.104312,15.713184
1995-01-05,15.580811,10.771054,13.711286,14.102115,16.990678,15.437414,15.307973,14.578987,14.405497,15.621471


<a id="plot2"></a>
### Plot 2 - Top 10 regions over time

In [54]:
plt = bokeh.charts.TimeSeries(top_regions_index, plot_width=800,legend=True, ylabel='Return')
show(plt)

<a id="scen2"></a>
## Regression - Average Return 

### Calculate slope of index by region

In [58]:
grouped = df.groupby("regionname")
slopes = []
for name, group in grouped:
    slope, intercept, r_value, p_value, std_err = stats.linregress(group.index, group["index"])
    slopes.append((name,slope))
slopedf = pd.DataFrame(slopes, columns=["regionname", "return"])
slopedf.sort_values("return", ascending=False).head()

,regionname,return
320,Shetland Islands,0.416367
162,Gwynedd,0.390494
192,Isle of Anglesey,0.386215
281,Pendle,0.381856
297,Richmondshire,0.380395


In [59]:
ret_vol = volatility.merge(slopedf, left_on="regionname", right_on="regionname",
                                      suffixes=("_std","_return"))

ret_vol = ret_vol[["regionname", "stdev", "return"]]
ret_vol = ret_vol.rename(columns={"stdev":"volatility"})
ret_vol.head()

,regionname,volatility,return
0,Aberdeenshire,16.742720,0.343023
1,Adur,26.851065,0.347008
2,Allerdale,30.943465,0.378900
3,Amber Valley,28.439592,0.347020
4,Angus,14.303827,0.243568


<a id="scen2plot1"></a>
### Plot 1

In [60]:
hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("desc", "@desc"),
        ])
source = ColumnDataSource(
        data=dict(
            x=ret_vol["volatility"],
            y=ret_vol["return"],
            desc=ret_vol["regionname"]
        )
    )

p = figure(plot_width=800, plot_height=600, tools=[hover])

p.xaxis.axis_label = "Risk (std dev)"
p.yaxis.axis_label = "Return (%average return)"

p.circle('x', 'y', source=source)

show(p)

In [61]:
top_regions = slopedf.sort_values("return", ascending=False).iloc[:10,0]
top_regions_data = df[df["regionname"].isin(top_regions)]
top_regions_index = top_regions_data.pivot(index="date",columns="regionname", values="index")
top_regions_index.head()

regionname,Allerdale,Barrow-in-Furness,Cornwall,Eden,Gwynedd,Isle of Anglesey,Manchester,Pendle,Richmondshire,Shetland Islands
date,,,,,,,,,,
1995-01-01,32.815689,27.375772,23.542952,30.524453,28.905250,28.657708,26.232063,32.329024,29.723110,NaN
1995-01-02,32.373205,30.138214,24.030445,30.807159,28.762934,27.071917,25.943357,32.223873,29.598240,NaN
1995-01-03,32.408196,30.269513,24.086569,29.857451,29.156520,26.427059,25.861962,32.557494,29.499071,NaN
1995-01-04,32.967298,31.979526,24.295812,29.372619,28.853126,24.400362,25.651124,32.415049,32.062258,NaN
1995-01-05,33.649422,30.697270,24.395658,28.989256,28.257518,24.533819,25.684809,32.963842,35.284189,NaN


<a id="scen2plot2"></a>
### Plot 2 - Top 10 Performers over time

In [65]:
plt = bokeh.charts.TimeSeries(top_regions_index, plot_width=800,legend=True)
show(plt)